In [1]:
# pip install chess.com
# https://www.youtube.com/watch?v=KYNbHGs-qG4
# pip install parsita


In [2]:
from chessdotcom import get_player_profile, get_player_stats, get_player_game_archives
import requests
import pprint
import pandas as pd
from parsita import *
from parsita.util import constant
import json
import chess.pgn
import io
import ftplib
import os
import logging

# https://pypi.org/project/pgn2data/
from converter.pgn_data import PGNData

In [3]:
# make json data easier to read
pp = pprint.PrettyPrinter(width=41, compact=True)

def get_user_archives(username, months):
    """
    get archive monthly files of specific chess.com player
    input:
    username - username of the chess.com player
    months - target months that we want to get the archives
    
    output:
    target_month - files of archives according to months parameter
    """
    archives = get_player_game_archives(username).json['archives']
    target_month = []
    for archive in archives:
        if archive[-7:] in months:
            target_month.append(archive)
    return target_month
    #most_recent_month_url = data['archives'][-1]
    #games = requests.get(most_recent_month_url).json()
    #return games

files = get_user_archives("tianminlyu",["2023/01"])


In [4]:
def get_archive_games(filename):
    """
    return games in one archive file
    
    input:
    filename - filename that contains game urls
    
    output: 
    """
    games = requests.get(filename).json()['games']
    return games

games = get_archive_games(files[-1])

In [5]:
files

['https://api.chess.com/pub/player/tianminlyu/games/2023/01']

In [6]:
def game_df(username,files):
    """
    import data from archive files and turn relevant data parameters into data frames
    
    input:
    username - username of the player
    files - file archived of the player
    
    output:
    a dataframe contains wanted columns
                        'username',
                         'urls', 
                         'time_control',
                        'end_time',
                        'uuid',
                        'initial_setup',
                        'time_class',
                        'rules',
                        'white_rating',
                        'white_username',
                        'black_rating',
                        'black_username',
                        'Event',
                        'Site',
                        'Date',
                        'Round',
                        'Result',
                        'CurrentPosition',
                        'ECO',
                        'ECOUrl',
                        'EndDate',
                        'EndTime',
                        'StartTime',
                        'Termination',
                        'Timezone',
                        'UTCDate',
                        'UTCTime',                        
                        
    """
    print("Player " + username + " is processing...")
    usernames = []
    urls = []
    time_control = [] 
    end_time = [] 
    uuid = []
    initial_setup = [] 
    time_class = [] 
    rules = [] 
    white_rating = [] 
    white_username = [] 
    black_rating = [] 
    black_username = []
    pgn = []
    event = []
    Site = []
    Date = []
    Round = []
    Result = []
    CurrentPosition = []
    ECO = []
    ECOUrl = []
    EndDate = []
    EndTime = []
    StartTime = []
    Termination = []
    Timezone = []
    UTCDate = []
    UTCTime = []
    
    for file in files:
        print(file + " " + " is processing...")
        games = get_archive_games(file)
        for game in games:
            try:
                usernames.append(username)
                urls.append(game['url'])
                time_control.append(game['time_control'])
                end_time.append(game['end_time'])
                uuid.append(game['uuid'])
                initial_setup.append(game['initial_setup'])
                time_class.append(game['time_class'])
                rules.append(game['rules'])
                white_rating.append(game['white']['rating'])
                white_username.append(game['white']['username'])
                black_rating.append(game['black']['rating'])
                black_username.append(game['black']['username'])
                pgn.append(game['pgn'])
                
                pgn_written = io.StringIO(game['pgn'])
                game_data = chess.pgn.read_game(pgn_written)
                event.append(game_data.headers['Event'])
                Site.append(game_data.headers['Site'])
                Date.append(game_data.headers['Date'])
                Round.append(game_data.headers['Round'])
                Result.append(game_data.headers['Result'])
                CurrentPosition.append(game_data.headers['CurrentPosition'])
                ECO.append(game_data.headers['ECO'])
                ECOUrl.append(game_data.headers['ECOUrl'])
                EndDate.append(game_data.headers['EndDate'])
                EndTime.append(game_data.headers['EndTime'])
                StartTime.append(game_data.headers['StartTime'])
                Termination.append(game_data.headers['Termination'])
                Timezone.append(game_data.headers['Timezone'])
                UTCDate.append(game_data.headers['UTCDate'])
                UTCTime.append(game_data.headers['UTCTime'])
            except Exception as e:
                print(e)
                print(game['url'])
    
    print("data fetch work is done.")
    
    df = pd.DataFrame(list(zip(usernames,
                               urls, 
                           time_control,
                          end_time,
                           uuid,
                           initial_setup,
                           time_class,
                           rules,
                           white_rating,
                           white_username,
                           black_rating,
                           black_username,
                           pgn,
                           event,
                          Site,
                          Date,
                          Round,
                          Result,
                          CurrentPosition,
                          ECO,
                          ECOUrl,
                          EndDate,
                          EndTime,
                          StartTime,
                          Termination,
                          Timezone,
                          UTCDate,
                          UTCTime)),
               columns =['username',
                         'urls', 
                         'time_control',
                        'end_time',
                        'uuid',
                        'initial_setup',
                        'time_class',
                        'rules',
                        'white_rating',
                        'white_username',
                        'black_rating',
                        'black_username',
                         'pgn',
                         'event',
                         'Site',
                          'Date',
                          'Round',
                          'Result',
                          'CurrentPosition',
                          'ECO',
                          'ECOUrl',
                          'EndDate',
                          'EndTime',
                          'StartTime',
                          'Termination',
                          'Timezone',
                          'UTCDate',
                          'UTCTime'
                        ])
    print("dataframe importing is done.")
    return df
    
data = game_df('tianminlyu',files)


Player tianminlyu is processing...
https://api.chess.com/pub/player/tianminlyu/games/2023/01  is processing...
'ECO'
https://www.chess.com/game/live/67392681337
data fetch work is done.
dataframe importing is done.


In [7]:
data

,username,urls,time_control,end_time,uuid,initial_setup,time_class,rules,white_rating,white_username,...,CurrentPosition,ECO,ECOUrl,EndDate,EndTime,StartTime,Termination,Timezone,UTCDate,UTCTime
0,tianminlyu,https://www.chess.com/game/live/66300097361,180+2,1672561823,414e47fc-89ad-11ed-b5e4-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,blitz,chess,2027,Kari66,...,4k3/4P3/4K3/8/8/8/8/8 b - -,A00,https://www.chess.com/openings/Polish-Opening-...,2023.01.01,08:30:23,08:21:34,Game drawn by stalemate,UTC,2023.01.01,08:21:34
1,tianminlyu,https://www.chess.com/game/live/66303703033,180+2,1672565229,96ddd677-89b5-11ed-b5e4-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,blitz,chess,2133,tianminlyu,...,8/1p6/8/P3k2P/2r1p3/4K3/8/8 w - -,A05,https://www.chess.com/openings/Reti-Opening-Ki...,2023.01.01,09:27:09,09:21:00,Kantemir-08 won by resignation,UTC,2023.01.01,09:21:00
2,tianminlyu,https://www.chess.com/game/live/66312643063,180+2,1672574060,dcacd72c-89c9-11ed-b5e4-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,blitz,chess,2168,Gloumir,...,4kn2/7p/1R4p1/4N1P1/2P1Pp2/5P2/6P1/6K1 b - -,A05,https://www.chess.com/openings/Reti-Opening-1....,2023.01.01,11:54:20,11:46:15,Gloumir won by resignation,UTC,2023.01.01,11:46:15
3,tianminlyu,https://www.chess.com/game/live/66313215887,180+2,1672574746,09570e3a-89cb-11ed-b5e4-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,blitz,chess,2101,adamchuk88,...,2R5/6Pk/1r1PKP2/1P6/8/8/8/8 b - -,A00,https://www.chess.com/openings/Barnes-Opening,2023.01.01,12:05:46,11:54:24,adamchuk88 won by resignation,UTC,2023.01.01,11:54:24
4,tianminlyu,https://www.chess.com/game/live/66318077641,180+2,1672579557,a3b44422-89d6-11ed-b5e4-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,blitz,chess,2105,tianminlyu,...,8/8/8/8/8/K7/3k4/2q5 w - -,A07,https://www.chess.com/openings/Kings-Indian-At...,2023.01.01,13:25:57,13:17:27,maxifargo won by resignation,UTC,2023.01.01,13:17:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,tianminlyu,https://www.chess.com/game/live/68301920833,180+2,1674563425,dd4273d0-9be1-11ed-bcea-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,blitz,chess,2010,tianminlyu,...,1k1r3r/pp3pp1/2p3b1/1P5p/Q1PN2B1/4P2P/P3Pb2/R1...,B52,https://www.chess.com/openings/Sicilian-Defens...,2023.01.24,15:46:42,15:29:27,Veerpatel2406 won by resignation,UTC,2023.01.24,15:29:27
73,tianminlyu,https://www.chess.com/game/live/68313171497,900+10,1674575202,cca11a8e-9bfb-11ed-bcea-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rapid,chess,1892,Veerpatel2406,...,8/pp1r2p1/4k3/6B1/5P1P/P2p1K2/1P1R4/8 b - -,B30,https://www.chess.com/openings/Sicilian-Defens...,2023.01.25,23:00:31,22:58:30,gmgera45 won by resignation,UTC,2023.01.25,22:58:30
74,tianminlyu,https://www.chess.com/game/live/68426573809,180+2,1674687631,c5e030b4-9d03-11ed-bcea-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,blitz,chess,1908,gmgera45,...,r1b1Nrk1/1p3pbp/p1n2pp1/1N6/4P3/Q7/PPP2PPP/2KR...,B20,https://www.chess.com/openings/Sicilian-Defens...,2023.01.26,15:54:51,15:27:03,tianminlyu won by checkmate,UTC,2023.01.26,15:27:03
75,tianminlyu,https://www.chess.com/game/live/68485960171,900+10,1674748491,e1c46f8a-9d8d-11ed-bcea-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rapid,chess,1618,Hornblower8,...,6k1/1Q6/p4pp1/2p4r/P1Pp4/1P4PK/2P1q2b/2B2R2 w - -,A04,https://www.chess.com/openings/Reti-Opening-Qu...,2023.01.26,16:23:25,16:04:55,tianminlyu won by resignation,UTC,2023.01.26,16:04:55


In [8]:
pgn_game = data['pgn'][0]


In [9]:
pgn_game

'[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2023.01.01"]\n[Round "-"]\n[White "Kari66"]\n[Black "tianminlyu"]\n[Result "1/2-1/2"]\n[CurrentPosition "4k3/4P3/4K3/8/8/8/8/8 b - -"]\n[Timezone "UTC"]\n[ECO "A00"]\n[ECOUrl "https://www.chess.com/openings/Polish-Opening-Kings-Indian-Defense"]\n[UTCDate "2023.01.01"]\n[UTCTime "08:21:34"]\n[WhiteElo "2027"]\n[BlackElo "2143"]\n[TimeControl "180+2"]\n[Termination "Game drawn by stalemate"]\n[StartTime "08:21:34"]\n[EndDate "2023.01.01"]\n[EndTime "08:30:23"]\n[Link "https://www.chess.com/game/live/66300097361"]\n\n1. b4 {[%clk 0:03:02]} 1... Nf6 {[%clk 0:03:00.6]} 2. Bb2 {[%clk 0:03:02.2]} 2... g6 {[%clk 0:03:02.1]} 3. a3 {[%clk 0:03:01.1]} 3... Bg7 {[%clk 0:03:03.4]} 4. c4 {[%clk 0:02:54.8]} 4... O-O {[%clk 0:03:04.4]} 5. e3 {[%clk 0:02:52.3]} 5... c6 {[%clk 0:03:05.6]} 6. Nh3 {[%clk 0:02:40.3]} 6... d5 {[%clk 0:03:06.6]} 7. d4 {[%clk 0:02:24]} 7... Bxh3 {[%clk 0:03:06.5]} 8. gxh3 {[%clk 0:02:24.6]} 8... Nbd7 {[%clk 0:03:03.2]} 9. Nd2 

In [11]:
BO_players = [
'AGcuber19',
'TLPAWN',
'xiaoanwu',
'EmmaXLi',
'akfunchess66',
'Marsboom', 
'Claraqiu',
'Ravenclawfairy', 
'Zora_zhu',
'BurleyWalrus']

# source https://docs.google.com/spreadsheets/d/1sLBU1ZgbubwXQSo9uBfoYJyEx0J7hKbKoLQBukbWOA8/edit#gid=0
# all RCC 2023 spring students
rcc_students = ['Chloe181920',
'achess222',
'benben14', 
'BrainOnFire123',
'CristenChess2012',
'minxingyang', 
'yangyangshen',
'Liam_snaperdragon',
'jerryzhao1',
'ChelseaFang',
'EricTang06',
'larryzh',
'oliver2033',
'AGcuber19',
'Cakieviv',
'1234Eric',
'Kara0523',
'Eason _Z',
'PeterLiu806',
'booklibh2',
'hantianlu',
'Jayden15908',
'hanwenwlu',
'Kayson2029',
'maxsufeng',
'Terrence_Mao',
'Owentx', 
'longfifthtie',
'BlueMagicalEarth',
'Xinran Yi',
'vfan16',
'taionemm',
'wenwen.kids@gmail.com',
'Will101916',
'milawang',
'zukinizap',
'yoyomandy',
'PromotionMay',
'aidenxzhou',
'AydenL09',
'countryballwarrior53428',
'japanball55555',
'TLPAWN',
'ethanlu001',
'xiaoanwu',
'canny9457',
'syuanlucky@gmail.com',
'Colsonlee',
'sheldon0410',
'SkyBlueRainbow',
'EmmaXLi',
'zmyg666',
'Pinkypinky2020',
'Chenyuan0',
'WildVividArch',
'Wildlunch',
'helloYB',
'StanLi100',
'superdupercoolguy',
'EmmaPZ',
'coco3139',
'Alicexh',
'xiaodolphin',
'jerrygordonli',
'Pi_Hu',
'Shinyangli', 
'WillQian2014',
'everyonespal',
'GracieYan',
'AnnieWang2014',
'Hugeggshell',
'darkyoungbus',
'unicornpancake99',
'Brewhaha2008',
'limefizzybugle',
'JoshuaGold',
'yaoamy',
'IrisWu2016',
'Annaqueen15 ',
'EmilyTang2017',
'Bigopenplan',
'Sneakycat123',
'magic_KQRBNP',
'dyrlxwu',
'augustinewz',
'louisz_chess',
'Udiedezly',
'Moneyllama7',
'Paleyoungwhale ',
'Ziqian_Gao',
'Wangleelee',
'summ3rr926',
'Billy520',
'LilliH',
'booklibh',
'RyderXW',
'NolieZ',
'akfunchess66',
'PinkPrimaryTeam',
'Stockfish14NNUE2000',
'Vovovopi',
'rosiezeng',
'Fulllearnedbroom',
'Chesslovereh6',
'Chessloverah6',
'Marsboom ',
'MelindaAndTofu',
'Matseptember',
'Florencexzx',
'Claraqiu',
'Jeremyjojo123',
'chessisbest312',
'oscarzhang818',
'chesskis_rui',
'Nolan330',
'kq136731',
'2016David',
'Leqih2013',
'MichaelH2021',
'duyuan',
'rrzhou',
'Hardygu666',
'abnwu',
'KatieL21',
'Universecluster',
'ChessmasterVH ',
'Aidenli2021',
'Ayden_WWW',
'sandyming',
'Sharkattackchess',
'Michael1628',
'Aaronxiu',
'Maxwelltian ',
'DavidL_Knight',
'tamesnazzyfish',
'SlimeRancher999 ',
'xiadi',
'VincentL2014',
'StellaL2012',
'Maxshen2015',
'DBBryceC',
'LeroyChang',
'Qangwchess',
'Jie1015',
'VincentC007',
'liujiawei',
'Cindycindy8',
'LinaEconomou',
'AZD15',
'Andrewc1125',
'Ethan765432',
'jasonwang21',
'Chessplayerbowser',
'shangshang0415',
'KateL1',
'ReachH42',
'EthanZ0209',
'Knight14000',
'andyzhon',
'Aiden_Chen',
'Pinhanggao',
'idiot10101',
'Kevinliu14',
'archer_feng',
'Accelerated-Dragon123',
'ElizabethGX',
'GoldenEagle2020',
'littlegrapewa',
'Williamchessfun',
'Hollowbill6',
'WilliamYun123',
'calmgiftedgoggles',
'Juliuslee123',
'a1928374655',
'Robotliam',
'Jaydenchess1234',
'victorvivi',
'ethanallen0',
'Ethanlizhao',
'EthanLZC',
'zenjiang ',
'nici_ec',
'maxweiWI',
'Snowman1119 ',
'foufou10',
'CJ-2015',
'xzlin',
'bo-yan',
'NewSavoryCrow',
'AmyLoveChess',
'bowenyang2013',
'suprib0414',
'Aarondestroy ',
'Haoran_Li',
'Error_PlayerNotFound',
'zitian',
'WILLIAM1109',
'alice66627',
'jaceliu',
'Tigertigeetigo',
'Cutepositivebit',
'HarryChess2023',
'TiffanyChess2023',
'Ryan0122',
'FlamingGodzilla',
'Aidendong ',
'HappyHappy101',
'JohnJ0125',
'Miya_Li',
'Emmaturtle17',
'ericbike',
'DavidTH2014',
'supermatthew615',
'HorseyTakesOver',
'FierceGiftedDragon',
'superking32',
'Chessgoodseed',
'Ravenclawfairy ',
'ripiteiiv',
'derderderderderderderder',
'alanzhuoxu',
'yo lan da 14111',
'10Chen',
'mophance',
'AdrianCharger',
'StellaXin',
'qbbandrew',
'AnyaWang',
'yaohengli',
'VictoriaYP',
'ElsaApplePeach26',
'Andrewx321',
'XinyiZ',
'Jesse1215',
'tangmingzhen',
'hellowiki08',
'happyac131',
'Brianhii',
'GoldenExcalibur',
'FirelionA',
'AaronYe123',
'BlakeZZB',
'Aliciagaomm',
'rongyujiang',
'ArthurJin',
'BlakeWu',
'Charlieheshi ',
'sxz109120@gmail.com',
'Fennyhan',
'Huaxu1981@gmail.com',
'Liam2017',
'joyhill2021',
'DerekCao17',
'Ndwyuan',
'Elizabethzhou2018',
'shiluqing1989',
'CharlieYZ',
'WilliamXuan',
'LegoManiacT',
'Maxiebaxie ',
'FFBROTHERSJR',
'FFBROTHERS',
'Williamzhou2016',
'Enoch1497',
'Diabol2020',
'IAMVERYCOOLANDAMAZING',
'skyfox07',
'Hvwei',
'Richard7773',
'vime735',
'emilyjiang0629',
'Super_tyu',
'N_Yu',
'matthew141118',
'Thomas-Feng',
'emmayang88',
'George-Hu',
'dustylaptop1',
'victoryunlong',
'Goldlivelymagician ',
'2019_david_zeng',
'YuxiQ',
'MatthewYangSCP',
'win1031',
'wiltonwang',
'Microsoft88888',
'ianwuchess',
'Modernbrake1 ',
'Tentacles777',
'oliviachesskid',
'NickZhu31',
'Leahyang0713',
'Harveywu',
'Whale88521',
'ConanKX',
'Derek2020',
'Haroldk10 ',
'mengrongliu',
'haha157',
'ls891',
'Lyy_chess',
'andrew1843',
'CommunistCannibal',
'rarepreparedgumdrop',
'Dou2014',
'Awesome_James',
'Hua2021',
'VVTOP ',
'Hydra123456',
'Griffin135246',
'EdwardShi',
'Pawkel99',
'Gildawarrior ',
'GoodSlimyCrystal',
'Xostar',
'OCAiden30044',
'bearbrandon',
'sweatyrhino',
'Turtleman26',
'Lisawang1018',
'Luke-Han',
'JaggerXie',
'sidneywei',
'oneapple396',
'sakuragl',
'OliviaOliviaQueen',
'ezez927 ',
'Super-Alex',
'ShawnShi2022',
'cloveriver',
'Jayden0128',
'TinyImageSYZ ',
'Thecats3rdlive',
'thinablepear',
'FunWiseHero',
'IvorC2',
'Dudutx',
'MinruiSun',
'yszhu',
'TiantianZhai',
'FlameJay19',
'Fengguoxinxin@gmail.com',
'Corey',
'andrewlovechess',
'HenryWen1',
'Az2116',
'AlexLi10',
'Doris150227',
'August2022W! ',
'Firmblade2016',
'Greeno_horse',
'lipengye',
'Bbq2101',
'Adayuanyin',
'AaronGYang',
'anthonyyu2014',
'aaronwang7',
'ArfyRuff',
'Minion47',
'BoredChipmunk84',
'Feifeigao',
'Aaronren',
'jw774',
'KomiCCE',
'Murieljiang ',
'LeonLi16',
'Indy501Chess',
'carolyn2014',
'Andy1223344',
'BBEJasperF',
'JessieY21',
'ekchess521',
'sophiehappy ',
'Ihaveryourcat',
'Sub2Djinxin',
'CaryCC',
'wisegeneralartist',
'lukewang1019',
'Supernana16',
'DanielMelbou',
'GraysonX2016',
'NathanBYWang',
'aaronliu16',
'tigeressyyy',
'BongDaiTeng',
'Brainycaptain',
'zoeychenyh',
'Blue raspberry667583',
'GavinTheKing668',
'Ilikeoctopuses ',
'Gettrain',
'Krcheng',
'Yuhong_LI',
'ronnieleng',
'DearBoldYouth',
'CuteSneakyRabbit',
'jecoliahx',
'EZSKplay',
'Kaikai2022',
'CL_LI',
'mlk3boys',
'Edwinyin',
'EddieLC',
'Katie1142 ',
'Chrisfei',
'Petrel2021',
'Zhihan_Zhao',
'Andrew_Y_Chen',
'galaxydragonet',
'lukaka122333',
'chesssophie24',
'IvorySwan7',
'BrianJ2016',
'MaxwellYin',
'Supermoon1',
'mo0601',
'DylanLi1024',
'EmilyLPC',
'BryanChao_Wingchess',
'Meizhou',
'nlwang',
'Wallacewang1214',
'Joychess18 ',
'KatelynT1614',
'JC_Chess01',
'Fearlessocelat14',
'Wilsonxiong',
'YONGXUAN2013',
'YONGJUN2013',
'GGCVC',
'IsabellaD2012',
'Emily0210',
'Floatinglizard1 ',
'kai20150909',
'Samh0101',
'Isaac_Rover',
'Jiayu13',
'Oweny954 ',
'Madsilkypiggy',
'Josie_rabbit',
'sneakyblackdragon ',
'TheGuardianofLight',
'Bryan830',
'qiful',
'ChessBeginner137',
'MasonEdamame',
'Wzhang2018',
'Cupcake_Boise',
'archesslover',
'junyiguo',
'jadenguo',
'averyzh',
'Bryant2014',
'Ryley_Houston ',
'Isaac_Houston',
'willhanzhu',
'SophiaZ2022',
'DanielXYChen ',
'jamemiddlebus',
'yushengp',
'Dogwater1012000',
'RareHandyCup',
'Liyuting',
'bluesplendidhat',
'HYW2021',
'Jonathanpower12',
'VincentYangChess',
'wqxu',
'OwenL5',
'Priscilla1203',
'jakel201',
'ninjapanda007',
'Dejia0419',
'happyjiaqi',
'Destroyer_Unstoppable251',
'angelawang2014',
'LauraW0115',
'lauratiffiany',
'Jaydencnc',
'Savedon37',
'Jeremyban',
'Andrew_Sun',
'P1GGU',
'LittleDolphin123',
'therealChloeW',
'vendis34',
'spreadtutoria ',
'Danieldong0619',
'Jacobfunny',
'HumanShadow',
'SapphireTiger10',
'Braveturtle20 ',
'ConnorC124641',
'ZZXatu',
'DiamondRain',
'Echessworld',
'Jefferyzg',
'GBD2015',
'ethanxue15',
'clora2000',
'JohnMirro111',
'MagKman2015',
'Purpleflower11',
'EvanY14',
'EllieY11',
'BenjaminChen2014',
'Wisedeal_14',
'PrestonZB',
'AriannaZB',
'zhongzian',
'MichaelWu18',
'Yicong26',
'vincentyli',
'Vivigymnast',
'yjmca',
'Myworld925',
'Clairelovescat',
'AlbertHW2016',
'JJXiong',
'Chengliam',
'newryland',
'Clairebeardare',
'KEVINC2016',
'AnnaJLi',
'Drywindytwig ',
'charlesliu20',
'Greenbecky6 ',
'lucas-2020',
'Aloofhawk',
'josephzczhang',
'Florencexzx',
'kkcaden',
'Bryan2014',
'magicfairybishop',
'Hilarioushousecat',
'Edward162105',
'Adam_GS',
'AliceCLi',
'ABChen',
'Zinaida7224',
'ThatSpicyArron',
'AudreyW2014',
'MightyChessYoda',
'wzl81561915',
'markhushen',
'George-555',
'EricHChess8015',
'OliviaQiChen',
'SuperEthanY',
'Cuteglitterunicorn',
'Masonthebacon2',
'yumitang',
'BurleyWalrus',
'richardsong101',
'GMYichenchess',
'56unicorn',
'Sploofy006',
'soccermaster3',
'drphilphdd',
'natureclub1000']

novice_player = [
    'DDTheoW',
'GibbsMei2023',
'Austinliu12',
'orcawes',
' tutuyellow',
'Wenwei Zhang',
'victoria_pan',
'Carolinelee2015',
'lukehuang2016',
'Benjamin0520',
'sebastianliang',
'miudu',
'caokancheng01',
'MotorcycleBatman',
'smartpikachu',
'RH1225',
'Kingsleyjiang0623',
'rebeccayguan',
'Anthonyj2014',
'ErikTep',
'AustinRen',
'Colby2016',
'AndyGG777',
'MightyMasonMan',
'YangAlvin',
'qishuai',
'jingyinghan',
'csdboy',
'KidsChessA',
'MoonyOwl',
'ruiruiplaychess',
'AidanSZhou',
'None',
'Emily1016',
'Firstarhxy',
'Audreylwlu',
'XIANGLI',
'sophialu518',
'huangsuustcgc',
'pokemonconnor',
'ChickenMaster1234567',
'kangaroo101',
'jiangzhizhi',
'Rockyco',
'AlexG0525',
'ChessAyla',
'Elaine201',
'liam_c_tan',
'trexwu',
'UnicornRebecca',
'wiliamgogo',
'XTheoKnight',
' GMD_2017',
'Ethantexan',
'dyytina',
'alvinzeng',
'GoGoBlue',
'WangNoah',
'Allison001',
'festus27',
'louisz_chess',
'rhettcheng',
'ALChessStory',
'DanielPlaytime ',
'mimiw01',
'EvanTeng',
'Cathye1',
'None',
'ArthurRocket',
'Tyzalex',
'ZhichengW',
'Brigittexie',
'angelazhangangelazhang',
'leopny',
'Quantum_Elephant',
'Catherine-Wu',
'Strawberrypizzaman',
'sycamoreyuan',
'Tiger520520',
'HenryLiu2016',
'Chloe051617',
'nolanyuan2015',
'HappyLion22',
'PriscillaLJ',
'VictoriaCui666',
'None',
'Ryanljl ',
'Xiaomanman',
'None',
'ZacharyQu',
'isaaclei123',
'MarcusChen2014 ',
'NathanChen_2016',
'YishanZ',
'henryk129',
'leolchess2022',
'OliviaL16925!',
'happysoph',
'None',
'cat24680',
'LittleWheats',
'audreyqueen ',
'Xuemeifeng',
'KevinMikey',
'Ethanyh ',
'IrisWu2016',
'fsypl009',
'EthanYY2015'
]

tianmin_players = {
    "BO" : ['AGcuber19',
            'TLPAWN',
            'xiaoanwu',
            'EmmaXLi',
            'akfunchess66',
            'Marsboom', 
            'Claraqiu',
            'Ravenclawfairy', 
            'Zora_zhu',
            'BurleyWalrus'],
    "BP" : ['taionemm',
            'augustinewz',
            'oscarzhang818',
            'yaohengli',
            'Wallacewang1214',
            'SophiaZ2022',
            'AliceCLi',
            'yumitang',
            'james2945',
            'Oinkoinkw'],
    "AN" : ['Cathye1',
            'lunathekitsune',
            'ArthurRocket',
            'vivianwwww20',
            'ChloeWang16',
            'Tyzalex',
            'ZhichengW',
            'Haochen1123',
            'jaydenlan0118',
            'ImRacoonie']
}

In [26]:
df_players = []
error_players = []

In [ ]:
for classes in tianmin_players.keys():
    for player in tianmin_players[classes]:
        try:
            files = get_user_archives(player,["2023/02","2023/01","2022/12"])
            df = game_df(player, files)
            df_players.append(df)
        except:
            print("This player account " + player + " does not exist")
            error_players.append(player)

Player AGcuber19 is processing...
https://api.chess.com/pub/player/agcuber19/games/2022/12  is processing...
'ECO'
https://www.chess.com/game/live/63766207495
'ECO'
https://www.chess.com/game/live/64002753283
'ECO'
https://www.chess.com/game/live/64089271325
https://api.chess.com/pub/player/agcuber19/games/2023/01  is processing...
'ECO'
https://www.chess.com/game/live/67449082651
https://api.chess.com/pub/player/agcuber19/games/2023/02  is processing...
'ECO'
https://www.chess.com/game/live/69024314043
'pgn'
https://www.chess.com/game/live/69027164491
'ECO'
https://www.chess.com/game/live/69027243575
data fetch work is done.
dataframe importing is done.
Player TLPAWN is processing...
https://api.chess.com/pub/player/tlpawn/games/2022/12  is processing...
'ECO'
https://www.chess.com/game/daily/456493725
https://api.chess.com/pub/player/tlpawn/games/2023/01  is processing...
'ECO'
https://www.chess.com/game/live/67387278311
data fetch work is done.
dataframe importing is done.
Player xi

In [ ]:
error_players

In [ ]:
print(len(error_players) * 100/ len(rcc_students))

In [ ]:
players_df = pd.concat(df_players)

In [ ]:
players_df.head()

In [ ]:
players_df.shape

In [ ]:
players_df['username'].nunique()

In [ ]:
def player_rating(row):  
    if row['username'].lower() == row['white_username'].lower():
        return row['white_rating']
    else:
        return row['black_rating']
    
players_df['player_rating'] = players_df.apply(lambda row: player_rating(row), axis=1)

In [ ]:
players_df['timestamp'] = pd.to_datetime(players_df['end_time'],unit='s')

In [ ]:
players_df.date = pd.to_datetime(players_df.timestamp)
players_df.set_index('timestamp',inplace=True)
players_df.head()

In [ ]:
players_df['year']=players_df.index.year
players_df['month']=players_df.index.month
players_df['day']=players_df.index.day
players_df.head()

In [ ]:
import plotly.express as px

fig = px.line(players_rapid_games.reset_index(), x="timestamp", y="player_rating", color = 'username')
fig.show()